In [ ]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/hyperparameter-tuning-cv/hyperparameter-tuning-cv.zip
!unzip -qq hyperparameter-tuning-cv.zip
%cd hyperparameter-tuning-cv

--2023-06-02 14:24:32--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/hyperparameter-tuning-cv/hyperparameter-tuning-cv.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 52.92.226.10, 52.92.145.90, 3.5.77.162, ...
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|52.92.226.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55556261 (53M) [binary/octet-stream]
Saving to: ‘hyperparameter-tuning-cv.zip’

hyperparameter-tuni 100%[===================>]  52.98M  23.0MB/s    in 2.3s    

2023-06-02 14:24:35 (23.0 MB/s) - ‘hyperparameter-tuning-cv.zip’ saved [55556261/55556261]

/content/hyperparameter-tuning-cv


In [ ]:
# import the necessary packages
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from skimage import feature
from imutils import paths
import numpy as np
import time
import cv2
import os

In [ ]:
class LocalBinaryPatterns:
  def __init__(self, numPoints, radius):
    self.numPoints = numPoints
    self.radius = radius 

  def describe(self, image, eps=1e-7):

    lbp = feature.local_binary_pattern(image, self.numPoints,
                                       self.radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
                             bins = np.arange(0, self.numPoints + 3),
                             range = (0, self.numPoints + 2))
    hist = hist.astype("float")
    hist /= (hist.sum()  + eps)

    return hist 

In [ ]:
args = {
	"dataset": "texture_dataset"
}

In [ ]:
imagePaths = list(paths.list_images(args['dataset']))

desc = LocalBinaryPatterns(24, 8)
data = []
labels = []

In [ ]:
for imagePath in imagePaths:
  image = cv2.imread(imagePath)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = desc.describe(gray)

  labels.append(imagePath.split(os.path.sep)[-2])

  data.append(hist)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
	random_state=22, test_size=0.25)

In [ ]:
parameters = [
	{"kernel":
		["linear"],
		"C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]},
	{"kernel":
		["poly"],
		"degree": [2, 3, 4],
		"C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]},
	{"kernel":
		["rbf"],
		"gamma": ["auto", "scale"],
		"C": [0.0001, 0.001, 0.1, 1, 10, 100, 1000]}
]

In [ ]:
grid = GridSearchCV(estimator=SVC(), param_grid=parameters, n_jobs=-1)
start = time.time()
grid.fit(trainX, trainY)
end = time.time()

# show the grid search information
print("[INFO] grid search took {:.2f} seconds".format(
	end - start))
print("[INFO] grid search best score: {:.2f}%".format(
	grid.best_score_ * 100))
print("[INFO] grid search best parameters: {}".format(
	grid.best_params_))

[INFO] grid search took 2.91 seconds
[INFO] grid search best score: 90.88%
[INFO] grid search best parameters: {'C': 1000, 'degree': 3, 'kernel': 'poly'}


In [ ]:
model = grid.best_estimator_
predictions = model.predict(testX)
print(classification_report(testY, predictions))

              precision    recall  f1-score   support

       brick       0.89      1.00      0.94         8
      marble       1.00      1.00      1.00         5
        sand       1.00      0.90      0.95        10

    accuracy                           0.96        23
   macro avg       0.96      0.97      0.96        23
weighted avg       0.96      0.96      0.96        23

